In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, silhouette_score,classification_report,precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel

In [2]:
data=pd.read_csv('D:\Data Science\Popular_Spotify_Songs.csv',encoding='latin1')

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Admin\AppData\Local\Temp\ipykernel_1828\1076136202.py:1: SyntaxWarning: invalid escape sequence '\D'
  data=pd.read_csv('D:\Data Science\Popular_Spotify_Songs.csv',encoding='latin1')


In [3]:
data.head()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6


In [4]:
data.tail()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
948,My Mind & Me,Selena Gomez,1,2022,11,3,953,0,91473363,61,...,144,A,Major,60,24,39,57,0,8,3
949,Bigger Than The Whole Sky,Taylor Swift,1,2022,10,21,1180,0,121871870,4,...,166,F#,Major,42,7,24,83,1,12,6
950,A Veces (feat. Feid),"Feid, Paulo Londra",2,2022,11,3,573,0,73513683,2,...,92,C#,Major,80,81,67,4,0,8,6
951,En La De Ella,"Feid, Sech, Jhayco",3,2022,10,20,1320,0,133895612,29,...,97,C#,Major,82,67,77,8,0,12,5
952,Alone,Burna Boy,1,2022,11,4,782,2,96007391,27,...,90,E,Minor,61,32,67,15,0,11,5


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    object
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    object
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      903 non-null    object
 14  bpm                   953 non-null    int64 
 15  key                   858 non-null    ob

In [6]:
data.describe()

,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,in_apple_playlists,in_apple_charts,in_deezer_charts,bpm,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
count,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000,953.00000,953.000000,953.000000,953.000000,953.000000,953.000000,953.000000
mean,1.556139,2018.238195,6.033578,13.930745,5200.124869,12.009444,67.812172,51.908709,2.666317,122.540399,66.96957,51.431270,64.279119,27.057712,1.581322,18.213012,10.131165
std,0.893044,11.116218,3.566435,9.201949,7897.608990,19.575992,86.441493,50.630241,6.035599,28.057802,14.63061,23.480632,16.550526,25.996077,8.409800,13.711223,9.912888
min,1.000000,1930.000000,1.000000,1.000000,31.000000,0.000000,0.000000,0.000000,0.000000,65.000000,23.00000,4.000000,9.000000,0.000000,0.000000,3.000000,2.000000
25%,1.000000,2020.000000,3.000000,6.000000,875.000000,0.000000,13.000000,7.000000,0.000000,100.000000,57.00000,32.000000,53.000000,6.000000,0.000000,10.000000,4.000000
50%,1.000000,2022.000000,6.000000,13.000000,2224.000000,3.000000,34.000000,38.000000,0.000000,121.000000,69.00000,51.000000,66.000000,18.000000,0.000000,12.000000,6.000000
75%,2.000000,2022.000000,9.000000,22.000000,5542.000000,16.000000,88.000000,87.000000,2.000000,140.000000,78.00000,70.000000,77.000000,43.000000,0.000000,24.000000,11.000000
max,8.000000,2023.000000,12.000000,31.000000,52898.000000,147.000000,672.000000,275.000000,58.000000,206.000000,96.00000,97.000000,97.000000,97.000000,91.000000,97.000000,64.000000


In [7]:
data.nunique()

track_name              943
artist(s)_name          645
artist_count              8
released_year            50
released_month           12
released_day             31
in_spotify_playlists    879
in_spotify_charts        82
streams                 949
in_apple_playlists      234
in_apple_charts         172
in_deezer_playlists     348
in_deezer_charts         34
in_shazam_charts        198
bpm                     124
key                      11
mode                      2
danceability_%           72
valence_%                94
energy_%                 80
acousticness_%           98
instrumentalness_%       39
liveness_%               68
speechiness_%            48
dtype: int64

In [8]:
data.shape

(953, 24)

In [9]:
data['streams'] = pd.to_numeric(data['streams'], errors='coerce')
data.dropna(subset=['streams'], inplace=True)
data["streams"] = data["streams"].astype(float)


In [10]:
data.isnull().sum()

track_name               0
artist(s)_name           0
artist_count             0
released_year            0
released_month           0
released_day             0
in_spotify_playlists     0
in_spotify_charts        0
streams                  0
in_apple_playlists       0
in_apple_charts          0
in_deezer_playlists      0
in_deezer_charts         0
in_shazam_charts        50
bpm                      0
key                     95
mode                     0
danceability_%           0
valence_%                0
energy_%                 0
acousticness_%           0
instrumentalness_%       0
liveness_%               0
speechiness_%            0
dtype: int64

In [42]:
#data.drop(columns='in_shazam_charts',inplace=True)

In [11]:
data['key'].fillna(data['key'].mode()[0], inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_1828\3855702249.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['key'].fillna(data['key'].mode()[0], inplace=True)


In [12]:
data['released_on'] = pd.to_datetime(data['released_year'].astype(str) + '-' + data['released_month'].astype(str) + '-' + data['released_day'].astype(str))
data['log_streams'] = np.log(data['streams'] + 1)

In [13]:
#data['released_on']=pd.to_datetime(dict(year=data['released_year'],month=data['released_month'],day=data['released_day']),errors='coerce')

#numeric_cols=['streams','bpm','danceability_%','valence_%','energy_%','acousticness_%','instrumentalness_%','liveness_%','speechiness_%']

In [14]:
def assign_popularity(log_streams):
    if log_streams <= 18.5:
        return 'Low'
    elif 18.5 < log_streams <= 19.5:
        return 'Medium'
    else:
        return 'High'

data['popularity_class'] = data['log_streams'].apply(assign_popularity)
data['song_age'] = (pd.Timestamp('now') - data['released_on']).dt.days // 365
data['is_colab'] = np.where(data['artist_count'] > 1, 1, 0)
data['energy_dance_ratio'] = data['energy_%'] / (data['danceability_%'] + 1e-6) # Adding a small constant to avoid division by zero
data['acoustic_vs_energy'] = data['acousticness_%'] - data['energy_%']

In [15]:
def categorize_bpm(bpm):
    if bpm < 100:
        return "Slow"
    elif 100 <= bpm < 120:
        return "Medium"
    elif 120 <= bpm < 140:
        return "Fast"
    else:
        return "Very Fast"

data['bpm_category'] = data['bpm'].apply(categorize_bpm)

In [16]:
print(data.columns)

Index(['track_name', 'artist(s)_name', 'artist_count', 'released_year',
       'released_month', 'released_day', 'in_spotify_playlists',
       'in_spotify_charts', 'streams', 'in_apple_playlists', 'in_apple_charts',
       'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts', 'bpm',
       'key', 'mode', 'danceability_%', 'valence_%', 'energy_%',
       'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%',
       'released_on', 'log_streams', 'popularity_class', 'song_age',
       'is_colab', 'energy_dance_ratio', 'acoustic_vs_energy', 'bpm_category'],
      dtype='object')


In [21]:
data['release_season'] = data['released_month'].apply(lambda x: 'Spring' if 3 <= x <= 5 else ('Summer' if 6 <= x <=8 else ('Autumn' if 9 <= x <= 11 else 'Winter')))

#One Hot Encode categorical features prior to splitting the data.
categorical_cols = ['key', 'mode', 'bpm_category','release_season']
encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
encoded_data = encoder.fit_transform(data[categorical_cols])

encoded_feature_names = encoder.get_feature_names_out(categorical_cols)

encoded_df = pd.DataFrame(encoded_data,columns=encoded_feature_names)

numeric_cols = ['artist_count', 'released_year', 'released_month',
               'in_spotify_playlists', 'in_spotify_charts', 'streams',
               'in_apple_playlists', 'in_apple_charts', 'in_deezer_charts',
               'danceability_%', 'valence_%', 'energy_%', 'acousticness_%',
               'instrumentalness_%', 'liveness_%', 'speechiness_%', 'bpm','song_age', 'is_colab','acoustic_vs_energy','energy_dance_ratio']

# Scale numerical features using StandardScaler
X_numeric = data[numeric_cols].fillna(data[numeric_cols].mean()) #fill the missing values with the mean. 
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)
X_scaled_df = pd.DataFrame(X_scaled, columns = numeric_cols)  # Convert back to DataFrame


X = pd.concat([X_scaled_df,encoded_df],axis = 1)
y = data['popularity_class']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) #keep the ratios

# Feature Selection SelectFromModel & Random Forest to select.
rf_classfier = RandomForestClassifier(n_estimators=100, random_state = 42)
rf_classfier.fit(X_train, y_train)

selector = SelectFromModel(rf_classfier, prefit = True, threshold = '0.5*mean') #Set the threshold

X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

print("Shape of X_train before feature selection:", X_train.shape)
print("Shape of X_train after feature selection:", X_train_selected.shape)


# Tune best Model : XGBoost
param_grid = {
    'n_estimators': [200, 300],
    'learning_rate': [0.01, 0.05],
    'max_depth': [3, 4],
    'subsample': [0.8, 0.7],
    'colsample_bytree': [0.9, 0.8],
    'gamma': [0, 0.1],
}

#Use k-fold stratified Cross Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) #important to set stratified! will not select well otherwise.

xgb = XGBClassifier(objective = 'multi:softmax', num_class=len(data['popularity_class'].unique()), eval_metric='mlogloss', use_label_encoder=False, random_state=42) #objective
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, scoring='accuracy', cv=skf, verbose=1, n_jobs=-1) #verbose set to see parameters

grid_search.fit(X_train_selected, y_train)

print("Best parameters:", grid_search.best_params_)

# Evaluate the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

<IPython.core.display.Javascript object>

C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Shape of X_train before feature selection: (761, 42)
Shape of X_train after feature selection: (761, 18)
Fitting 5 folds for each of 64 candidates, totalling 320 fits


C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:09:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best parameters: {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
Accuracy: 0.9947643979057592
              precision    recall  f1-score   support

        High       1.00      1.00      1.00        95
         Low       1.00      0.97      0.98        33
      Medium       0.98      1.00      0.99        63

    accuracy                           0.99       191
   macro avg       0.99      0.99      0.99       191
weighted avg       0.99      0.99      0.99       191



## PREDICTIVE MODELING

In [22]:
feature_cols = ['artist_count', 'released_year', 'released_month',
               'in_spotify_playlists', 'in_spotify_charts',
               'in_apple_playlists', 'in_apple_charts', 'in_deezer_charts',
               'danceability_%', 'valence_%', 'energy_%', 'acousticness_%',
               'instrumentalness_%', 'liveness_%', 'speechiness_%', 'bpm']

In [23]:
data = pd.get_dummies(data, columns=['key', 'mode', 'bpm_category'], drop_first=True)

In [24]:
X = data[feature_cols]
y = data['popularity_class']

In [25]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')  # Replace missing values with the mean
X = imputer.fit_transform(X)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [27]:
from sklearn.tree import DecisionTreeClassifier

In [31]:
# model = LogisticRegression(random_state=42)
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)
# #------------------------------------
# dtc = DecisionTreeClassifier(random_state=42)

# # Define a range of hyperparameters to tune
# param_grid = {
#     'max_depth': [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
#     'min_samples_split': [2, 5, 10],
# }

# # Use GridSearchCV for hyperparameter tuning and cross-validation
# grid_search = GridSearchCV(dtc, param_grid, cv=5, scoring='accuracy')
# grid_search.fit(X_train, y_train)

# # Best parameters from Grid Search
# best_params = grid_search.best_params_
# print(f"Best parameters: {best_params}")

# # Train the model with the best parameters
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(X_test)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:.2f}")


Best parameters: {'max_depth': 3, 'min_samples_split': 2}
Accuracy: 0.73


In [32]:
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

Accuracy: 0.7277486910994765


## ------- RECOMMENDATION  SYSTEM -------

In [33]:
def create_profile(data, artist_name):
    artist_songs = data[data['artist(s)_name'] == artist_name][feature_cols]  # Ensure feature_cols is defined
    if not artist_songs.empty:
        return artist_songs.mean().values.reshape(1, -1)
    else:
        return None

In [44]:
def recommend_songs(data, artist_name, top_n=5):
    profile = create_profile(data, artist_name)
    if profile is None:
        return "Artist not found or no songs available."


    # Handle missing values in feature columns using SimpleImputer
    imputer = SimpleImputer(strategy='mean')
    data[feature_cols] = imputer.fit_transform(data[feature_cols])

    # Ensure profile has the same features
    song_features = data[feature_cols]

    # Scale the features
    scaler = StandardScaler()
    scaled_song_features = scaler.fit_transform(song_features)
    scaled_profile = scaler.transform(profile)

    # Calculate cosine similarity
    cosine_similarities = cosine_similarity(scaled_profile, scaled_song_features)
    
    # Get top songs based on similarity
    similar_songs_indices = cosine_similarities.argsort()[0][::-1]
    
    # Exclude songs by the input artist
    recommendations = []
    for index in similar_songs_indices:
        if data['artist(s)_name'].iloc[index] != artist_name:
            recommendations.append(data['track_name'].iloc[index] + " by " + data['artist(s)_name'].iloc[index])
        if len(recommendations) >= top_n:
            break
    
    if recommendations:
        return recommendations
    else:
        return "No recommendations found."

In [46]:
#artist name have to get from user 
print(data['artist(s)_name'].unique())

['Latto, Jung Kook' 'Myke Towers' 'Olivia Rodrigo' 'Taylor Swift'
 'Bad Bunny' 'Dave, Central Cee' 'Eslabon Armado, Peso Pluma' 'Quevedo'
 'Gunna' 'Peso Pluma, Yng Lvcas' 'Bad Bunny, Grupo Frontera' 'NewJeans'
 'Miley Cyrus' 'David Kushner' 'Harry Styles' 'SZA' 'Fifty Fifty'
 'Billie Eilish' 'Feid, Young Miko' 'Jimin'
 'Gabito Ballesteros, Junior H, Peso Pluma' 'Arctic Monkeys'
 'Bizarrap, Peso Pluma' 'The Weeknd, Madonna, Playboi Carti'
 'Fuerza Regida' 'Rï¿½ï¿½ma, Selena G' 'Tainy, Bad Bunny' 'Morgan Wallen'
 'Dua Lipa' 'Troye Sivan' 'Peso Pluma, Grupo Frontera'
 'The Weeknd, 21 Savage, Metro Boomin' 'Karol G, Shakira'
 'Big One, Duki, Lit Killah, Maria Becerra, FMK, Rusherking, Emilia, Tiago pzk'
 'Yahritza Y Su Esencia, Grupo Frontera' 'Junior H, Peso Pluma'
 'Post Malone, Swae Lee' 'Bebe Rexha, David Guetta'
 'Tyler, The Creator, Kali Uchis' 'Nicki Minaj, Aqua, Ice Spice'
 'OneRepublic' 'The Weeknd' 'The Weeknd, Daft Punk'
 'Ariana Grande, The Weeknd' 'Feid, Myke Towers, Sky Rompi

In [49]:
artist_name = "Raim Laode"  # Replace with artist's name
recommendations = recommend_songs(data, artist_name)
print(f"Recommendations for {artist_name}:")
print(recommendations,end='\n')



Recommendations for Raim Laode:
["You're On Your Own, Kid by Taylor Swift", 'My Mind & Me by Selena Gomez', 'Midnight Rain by Taylor Swift', 'Mastermind by Taylor Swift', 'Chance by Paulo Londra']


C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## ------- CLUSTERING SONGS --------

In [25]:
audio_features = ['danceability_%', 'energy_%', 'valence_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%', 'bpm']

In [26]:
X_cluster = data[audio_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

In [27]:
silhouette_scores = []
for n_clusters in range(2, 11):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_scaled)
    silhouette_avg = silhouette_score(X_scaled, cluster_labels)
    silhouette_scores.append(silhouette_avg)


C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  

In [32]:
optimal_n_clusters = range(3, 11)[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters: {optimal_n_clusters}")

Optimal number of clusters: 3


In [33]:
kmeans = KMeans(n_clusters=optimal_n_clusters, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

data['cluster'] = clusters

In [34]:
for cluster in range(optimal_n_clusters):
    cluster_data = data[data['cluster'] == cluster]
    print(f"Cluster {cluster}:")
    print(cluster_data[audio_features].mean())
    print("\n")

Cluster 0:
danceability_%         70.888489
energy_%               72.334532
valence_%              59.571942
acousticness_%         15.868705
instrumentalness_%      0.602518
liveness_%             19.447842
speechiness_%           6.827338
bpm                   121.284173
dtype: float64


Cluster 1:
danceability_%         54.799197
energy_%               47.144578
valence_%              33.706827
acousticness_%         52.373494
instrumentalness_%      4.662651
liveness_%             16.220884
speechiness_%           5.638554
bpm                   122.397590
dtype: float64


Cluster 2:
danceability_%         72.857143
energy_%               62.802721
valence_%              50.503401
acousticness_%         26.632653
instrumentalness_%      0.074830
liveness_%             16.925170
speechiness_%          30.285714
bpm                   127.619048
dtype: float64




In [57]:
data

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,in_shazam_charts,bpm,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,released_on,log_streams,popularity_class,song_age,is_colab,energy_dance_ratio,acoustic_vs_energy,key_A#,key_B,key_C#,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,mode_Minor,bpm_category_Medium,bpm_category_Slow,bpm_category_Very Fast,cluster
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2.0,2023.0,7.0,14,553.0,147.0,141381703.0,43.0,263.0,45,10.0,826,125.0,80.0,89.0,83.0,31.0,0.0,8.0,4.0,2023-07-14,18.766974,Medium,2,1,1.037500,-52,False,True,False,False,False,False,False,False,False,False,False,False,False,False,0
1,LALA,Myke Towers,1.0,2023.0,3.0,23,1474.0,48.0,133716286.0,48.0,126.0,58,14.0,382,92.0,71.0,61.0,74.0,7.0,0.0,10.0,4.0,2023-03-23,18.711231,Medium,2,0,1.042254,-67,False,False,True,False,False,False,False,False,False,False,False,False,True,False,0
2,vampire,Olivia Rodrigo,1.0,2023.0,6.0,30,1397.0,113.0,140003974.0,94.0,207.0,91,14.0,949,138.0,51.0,32.0,53.0,17.0,0.0,31.0,6.0,2023-06-30,18.757181,Medium,2,0,1.039216,-36,False,False,False,False,False,False,True,False,False,False,False,False,False,False,1
3,Cruel Summer,Taylor Swift,1.0,2019.0,8.0,23,7858.0,100.0,800840817.0,116.0,207.0,125,12.0,548,170.0,55.0,58.0,72.0,11.0,0.0,11.0,15.0,2019-08-23,20.501173,High,6,0,1.309091,-61,False,False,False,False,False,False,False,False,False,False,False,False,False,True,0
4,WHERE SHE GOES,Bad Bunny,1.0,2023.0,5.0,18,3133.0,50.0,303236322.0,84.0,133.0,87,15.0,425,144.0,65.0,23.0,80.0,14.0,63.0,11.0,6.0,2023-05-18,19.530023,High,2,0,1.230769,-66,False,False,False,False,False,False,False,False,False,False,True,False,False,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,My Mind & Me,Selena Gomez,1.0,2022.0,11.0,3,953.0,0.0,91473363.0,61.0,13.0,37,1.0,0,144.0,60.0,24.0,39.0,57.0,0.0,8.0,3.0,2022-11-03,18.331558,Low,2,0,0.650000,18,False,False,False,False,False,False,False,False,False,False,False,False,False,True,1
949,Bigger Than The Whole Sky,Taylor Swift,1.0,2022.0,10.0,21,1180.0,0.0,121871870.0,4.0,0.0,8,0.0,0,166.0,42.0,7.0,24.0,83.0,1.0,12.0,6.0,2022-10-21,18.618481,Medium,2,0,0.571429,59,False,False,False,False,False,False,False,True,False,False,False,False,False,True,1
950,A Veces (feat. Feid),"Feid, Paulo Londra",2.0,2022.0,11.0,3,573.0,0.0,73513683.0,2.0,0.0,7,0.0,0,92.0,80.0,81.0,67.0,4.0,0.0,8.0,6.0,2022-11-03,18.112982,Low,2,1,0.837500,-63,False,False,True,False,False,False,False,False,False,False,False,False,True,False,0
951,En La De Ella,"Feid, Sech, Jhayco",3.0,2022.0,10.0,20,1320.0,0.0,133895612.0,29.0,26.0,17,0.0,0,97.0,82.0,67.0,77.0,8.0,0.0,12.0,5.0,2022-10-20,18.712571,Medium,2,1,0.939024,-69,False,False,True,False,False,False,False,False,False,False,False,False,True,False,0
